<a href="https://colab.research.google.com/github/Thanveerahmd/Fyp_Violence_transformer/blob/main/RLVS_video_transformers_40_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
cd /content/gdrive/MyDrive/Final_Year_Project/Dataset/RLVS/real_life_violence_situations

/content/gdrive/MyDrive/Final_Year_Project/Dataset/RLVS/real_life_violence_situations


In [ ]:
ls

 animation.gif                  rlvs_train_data_40.npy
'Real Life Violence Dataset'/   rlvs_train_data.npy
 rlvs/                          rlvs_train_labels_40.npy
 rlvs_test_data_40.npy          rlvs_train_labels.npy
 rlvs_test_data.npy             rlvs_train.txt
 rlvs_test_labels_40.npy        test.csv
 rlvs_test_labels.npy           train.csv
 rlvs_test.txt


In [ ]:
!pip install -q git+https://github.com/tensorflow/docs

## Setup

In [ ]:
from tensorflow_docs.vis import embed
from tensorflow.keras import layers
from tensorflow import keras

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

## Define hyperparameters

In [ ]:
MAX_SEQ_LENGTH = 40
NUM_FEATURES = 1024
IMG_SIZE = 200

EPOCHS = 50

## Data preparation

We will mostly be following the same data preparation steps in this example, except for
the following changes:

* We reduce the image size to 128x128 instead of 224x224 to speed up computation.
* Instead of using a pre-trained [InceptionV3](https://arxiv.org/abs/1512.00567) network,
we use a pre-trained
[DenseNet121](http://openaccess.thecvf.com/content_cvpr_2017/papers/Huang_Densely_Connected_Convolutional_CVPR_2017_paper.pdf)
for feature extraction.
* We directly pad shorter videos to length `MAX_SEQ_LENGTH`.

First, let's load up the
[DataFrames](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).

In [ ]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

center_crop_layer = layers.CenterCrop(IMG_SIZE, IMG_SIZE)


def crop_center(frame):
    cropped = center_crop_layer(frame[None, ...])
    cropped = cropped.numpy().squeeze()
    return cropped


# Following method is modified from this tutorial:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
def load_video(path, max_frames=0):

    cap = cv2.VideoCapture(path)

    width  = cap.get(3)  # float `width`
    height = cap.get(4)  # float `height`
    print('width, height:', width, height)

    frames = []
    try:
        while True:

            ret, frame = cap.read()
            if not ret:
                break
            #frame = crop_center(frame)
            frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)


def build_feature_extractor():
    feature_extractor = keras.applications.DenseNet121(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.densenet.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()


# Label preprocessing with StringLookup.
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"]), mask_token=None
)
print(label_processor.get_vocabulary())


def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    labels = df["tag"].values
    labels = label_processor(labels[..., None]).numpy()

    # `frame_features` are what we will feed to our sequence model.
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))

        #print(path)
        print(frames.shape)

        # Pad shorter videos.
        if len(frames) < MAX_SEQ_LENGTH:
            diff = MAX_SEQ_LENGTH - len(frames)
            padding = np.zeros((diff, IMG_SIZE, IMG_SIZE, 3))
            frames = np.concatenate(frames, padding)

        frames = frames[None, ...]

        # Initialize placeholder to store the features of the current video.
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                if np.mean(batch[j, :]) > 0.0:
                    temp_frame_features[i, j, :] = feature_extractor.predict(
                        batch[None, j, :]
                    )

                else:
                    temp_frame_features[i, j, :] = 0.0

        frame_features[idx,] = temp_frame_features.squeeze()

    return frame_features, labels


Total videos for training: 1600
Total videos for testing: 400
29097984/29084464 [==============================] - 0s 0us/step
['NonViolence', 'Violence']


In [ ]:
train_data, train_labels = prepare_all_videos(train_df, "rlvs/train")
test_data, test_labels = prepare_all_videos(test_df, "rlvs/test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in test set: {test_data[1].shape}")

In [ ]:
from numpy import save

save('rlvs_train_data.npy', train_data)
save('rlvs_train_labels.npy', train_labels)
save('rlvs_test_data.npy', test_data)
save('rlvs_test_labels.npy', test_labels)

In [ ]:
train_data, train_labels = np.load("rlvs_train_data_40.npy"), np.load("rlvs_train_labels_40.npy")
test_data, test_labels = np.load("rlvs_test_data_40.npy"), np.load("rlvs_test_labels_40.npy")

In [ ]:
print(f"Frame features in train set: {train_data.shape}")
print(f"Frame features in train label set: {train_labels.shape}")
print(f"Frame features in test set: {test_data.shape}")
print(f"Frame features in test label set: {test_labels.shape}")

Frame features in train set: (1600, 40, 1024)
Frame features in train label set: (1600, 1)
Frame features in test set: (400, 40, 1024)
Frame features in test label set: (400, 1)


Calling `prepare_all_videos()` on `train_df` and `test_df` takes ~20 minutes to
complete. For this reason, to save time, here we download already preprocessed NumPy arrays:

## Building the Transformer-based model

We will be building on top of the code shared in
[this book chapter](https://livebook.manning.com/book/deep-learning-with-python-second-edition/chapter-11) of
[Deep Learning with Python (Second ed.)](https://www.manning.com/books/deep-learning-with-python)
by François Chollet.

First, self-attention layers that form the basic blocks of a Transformer are
order-agnostic. Since videos are ordered sequences of frames, we need our
Transformer model to take into account order information.
We do this via **positional encoding**.
We simply embed the positions of the frames present inside videos with an
[`Embedding` layer](https://keras.io/api/layers/core_layers/embedding). We then
add these positional embeddings to the precomputed CNN feature maps.

In [ ]:

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim
        )
        self.sequence_length = sequence_length
        self.output_dim = output_dim

    def call(self, inputs):
        # The inputs are of shape: `(batch_size, frames, num_features)`
        length = tf.shape(inputs)[1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_positions = self.position_embeddings(positions)
        return inputs + embedded_positions

    def compute_mask(self, inputs, mask=None):
        mask = tf.reduce_any(tf.cast(inputs, "bool"), axis=-1)
        return mask


Now, we can create a subclassed layer for the Transformer.

In [ ]:

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.3
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation=tf.nn.gelu), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]

        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)


In [ ]:
def get_compiled_model():
    sequence_length = MAX_SEQ_LENGTH
    embed_dim = NUM_FEATURES
    dense_dim = 64
    num_heads = 8
    classes = len(label_processor.get_vocabulary())

    inputs = keras.Input(shape=(None, None))
    x = PositionalEmbedding(
        sequence_length, embed_dim, name="frame_position_embedding"
    )(inputs)

    x = layers.Bidirectional(layers.LSTM(embed_dim,return_sequences=True))(x)
    x = TransformerEncoder(2*embed_dim, dense_dim, num_heads, name="transformer_layer_1")(x)
    #x = TransformerEncoder(2*embed_dim, dense_dim, num_heads, name="transformer_layer_2")(x)
    x = layers.Bidirectional(layers.LSTM(2*embed_dim,return_sequences=True))(x)
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dropout(0.5)(x)
    
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"]
    )
    return model

In [ ]:
model = get_compiled_model()

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, None, None)]      0         
_________________________________________________________________
frame_position_embedding (Po (None, None, 1024)        40960     
_________________________________________________________________
bidirectional_4 (Bidirection (None, None, 2048)        16785408  
_________________________________________________________________
transformer_layer_1 (Transfo (None, None, 2048)        134541376 
_________________________________________________________________
bidirectional_5 (Bidirection (None, None, 4096)        67125248  
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 4096)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 4096)              0   

## Utility functions for training

In [ ]:
def run_experiment():
    filepath = "/tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    
    history = model.fit(
        train_data,
        train_labels,
        validation_split=0.15,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    model.load_weights(filepath)
    _, accuracy = model.evaluate(test_data, test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return model,history


## Model training and inference

In [ ]:
trained_model,history = run_experiment()

Epoch 1/50
43/43 [==============================] - 71s 1s/step - loss: 1.2487 - accuracy: 0.5316 - val_loss: 1.0696 - val_accuracy: 0.0000e+00

Epoch 00001: val_loss improved from inf to 1.06964, saving model to /tmp/video_classifier
Epoch 2/50
43/43 [==============================] - 50s 1s/step - loss: 0.6855 - accuracy: 0.5772 - val_loss: 0.8440 - val_accuracy: 0.0000e+00

Epoch 00002: val_loss improved from 1.06964 to 0.84405, saving model to /tmp/video_classifier
Epoch 3/50
43/43 [==============================] - 50s 1s/step - loss: 0.6925 - accuracy: 0.5684 - val_loss: 0.8057 - val_accuracy: 0.0000e+00

Epoch 00003: val_loss improved from 0.84405 to 0.80567, saving model to /tmp/video_classifier
Epoch 4/50
43/43 [==============================] - 50s 1s/step - loss: 0.6926 - accuracy: 0.5596 - val_loss: 1.0490 - val_accuracy: 0.0000e+00

Epoch 00004: val_loss did not improve from 0.80567
Epoch 5/50
43/43 [==============================] - 50s 1s/step - loss: 0.6871 - accuracy: 

In [ ]:
from matplotlib import pyplot

In [ ]:
    pyplot.title('Learning Curves')
    pyplot.xlabel('Epoch')
    pyplot.ylabel('Root Mean Squared Error')
    pyplot.plot(history.history['loss'], label='train')
    pyplot.plot(history.history['val_loss'], label='val')
    pyplot.legend()
    pyplot.show()


In [ ]:
    pyplot.title('Learning Curves')
    pyplot.xlabel('Epoch')
    pyplot.ylabel('Root Mean Squared Error')
    pyplot.plot(history.history['accuracy'], label='train')
    pyplot.plot(history.history['val_accuracy'], label='val')
    pyplot.legend()
    pyplot.show()

**Note**: This model has ~4.23 Million parameters, which is way more than the sequence
model (99918 parameters) we used in the prequel of this example.  This kind of
Transformer model works best with a larger dataset and a longer pre-training schedule.

In [ ]:

def prepare_single_video(frames):
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    # Pad shorter videos.
    if len(frames) < MAX_SEQ_LENGTH:
        diff = MAX_SEQ_LENGTH - len(frames)
        padding = np.zeros((diff, IMG_SIZE, IMG_SIZE, 3))
        frames = np.concatenate(frames, padding)

    frames = frames[None, ...]

    # Extract features from the frames of the current video.
    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            if np.mean(batch[j, :]) > 0.0:
                frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
            else:
                frame_features[i, j, :] = 0.0

    return frame_features


def predict_action(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("rlvs/test/", path))
    frame_features = prepare_single_video(frames)
    probabilities = trained_model.predict(frame_features)[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames


# This utility is for visualization.
# Referenced from:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
def to_gif(images):
    converted_images = images.astype(np.uint8)
    imageio.mimsave("animation.gif", converted_images, fps=10)
    return embed.embed_file("animation.gif")


test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")
test_frames = predict_action(test_video)
to_gif(test_frames[:MAX_SEQ_LENGTH])

The performance of our model is far from optimal, because it was trained on a
small dataset.